In [140]:
import array, json, math, os, random
from xml.dom.minidom import parse
import xml.dom.minidom

#parameters
K = .04 #global bundling constant controlling edge stiffness
S_initial = .04 #initial distance to move points
C = 5 #number of cycles to perform
I_initial = 60 #initial number of iterations for cycle
I_rate = 0.6667 #rate at which the number of iterations decreases
epsilon = 1e-3 
compatibility_threshold = 0.6 #threshold for the compatibility score at which two edges are considered compatible
scaling_factor = .6 #factor the edges are scaled down at

#geometry methods
def dotProduct(point1,point2):
    return point1[0]*point2[0] + point1[1]*point2[1]

def euclideanDistance(point1,point2):
    return math.sqrt(math.pow(point2[0] - point1[0],2) + math.pow(point2[1] - point1[1],2))

def edgeLength(edge):
    (startX,startY) = edge[0]
    (endX,endY) = edge[1]
    return math.sqrt(math.pow(endX-startX,2) + math.pow(endY-startY,2))

def findMidpoint(point1,point2):
    startX = point1[0]
    startY = point1[1]
    endpointX = point2[0]
    endpointY = point2[1]
    midX = (startX + endpointX)/2
    midY = (startY + endpointY)/2
    return [midX,midY]

def projectPointOnLine(point,edge):
    L = math.sqrt(math.pow((edge[1][0] - edge[0][0]),2) + math.pow((edge[1][1] - edge[0][1]),2))
    r = ((edge[0][1] - point[1]) * (edge[0][1] - edge[1][1]) - (edge[0][0] - point[0]) * (edge[1][0] - edge[0][0])) / (L * L)
    return [(edge[0][0] + (r *(edge[1][0] - edge[0][0]))), (edge[0][1] + (r *(edge[1][1] - edge[0][1])))]

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

#force calculation methods
def calculateSpringForce(prevPoint,current,nextPoint,kp):
    xDistance = prevPoint[0] - current[0] +  nextPoint[0] - current[0]
    yDistance = prevPoint[1] - current[1] + nextPoint[1] - current[1]
    xForce = kp * xDistance
    yForce = kp * yDistance
    return([xForce,yForce])

def calculateElectroStaticForce(point1,point2):
    xDistance = point2[0] - point1[0]
    yDistance = point2[1] - point1[1]
    
    if abs(xDistance) > math.sqrt(epsilon):
        xForce = xDistance * 1/math.pow(edgeLength([point1,point2]),2)
    else:
        xForce = 0.1
    if abs(yDistance) > math.sqrt(epsilon):
        yForce = yDistance * 1/math.pow(edgeLength([point1,point2]),2)
    else:
        yForce = 0.1
    
    return [xForce,yForce]

def calculateTotalForces(prevPoint,current,nextPoint,i,kp):
    force = [0,0]
    (springX,springY) =  calculateSpringForce(prevPoint,current,nextPoint,kp)
    force[0] += springX
    force[1] += springY 
    for edgeNum in compatibilityList[i]:
        (electroX, electroY) = calculateElectroStaticForce(current,subDividedEdges[edgeNum][i])
        force[0] += electroX
        force[1] += electroY
    return force

#compatibility methods
def angleCompatibility(edge1,edge2):
    return abs(dotProduct([edge1[1][0] - edge1[0][0],edge1[1][1]-edge1[0][1]],[edge2[1][0] - edge2[0][0],edge2[1][1]-edge2[0][1]]) / (edgeLength(edge1) * edgeLength(edge2)))

def scaleCompatibility(edge1,edge2):
    averageLength = (edgeLength(edge1) + edgeLength(edge2))/2
    return 2 / (averageLength / min(edgeLength(edge1),edgeLength(edge2)) + max(edgeLength(edge1),edgeLength(edge2)) / averageLength)
def positionCompatibility(edge1,edge2):
    averageLength = (edgeLength(edge1) + edgeLength(edge2))/2
    midpointEdge1 = findMidpoint(edge1[0],edge1[1])
    midpointEdge2 = findMidpoint(edge2[0],edge2[1])
    return averageLength / (averageLength + euclideanDistance(midpointEdge1,midpointEdge2))
    
def edgeVisibility(edge1,edge2):
    I0 = projectPointOnLine(edge2[0],edge1)
    I1 = projectPointOnLine(edge2[1],edge1)
    midI = findMidpoint(I0,I1)
    midEdge1 = findMidpoint(edge1[0],edge1[1])
    if I0 != I1:
        return max(0,1 - 2 * euclideanDistance(midI,midEdge1) / euclideanDistance(I0,I1))
    else:
        return 0
    
def visibilityCompatibility(edge1,edge2):
    return min(edgeVisibility(edge1,edge2),edgeVisibility(edge2,edge1))

def compatibilityScore(edge1,edge2):
    return angleCompatibility(edge1,edge2) * scaleCompatibility(edge1,edge2) * positionCompatibility(edge1,edge2) * visibilityCompatibility(edge1,edge2)

def getGrayScale(val,maxVal,minVal):
    print 1

In [141]:
with open("../data/2014.json") as f:
    data = json.load(f)

In [147]:
edges = []
exportVals = []
for feature in data:
    startX  = feature['org_wm'][0]
    startY = feature['org_wm'][1]
    exportVal = feature['export_val']
    endX = feature['dst_wm'][0]
    endY = feature['dst_wm'][1]
    #eliminates single points 
    if startX != endX and startY != endY:
        #limitations on the edges to be considered
        if  100 < startX < 170  and 100 < endX < 170 and 0 < startY  < 100 and 0< endY < 100:
            edge = [[[startX/scaling_factor,startY/scaling_factor],[endX/scaling_factor,endY/scaling_factor]]]
            edges += edge
            exportVals += [exportVal]
       

In [148]:
#computes list of compatible edges
compatibilityList = []
for i in range(0, len(edges)):
    compatibilityList += [[]]
for i in range(0,len(edges) - 1):
    if i % 1000 == 0: print i
    for j in range(i + 1, len(edges)):
        if compatibilityScore(edges[i],edges[j]) >= compatibility_threshold:
            compatibilityList[i] += [j]
            compatibilityList[j] += [i]
            

0
1000


In [149]:
I = I_initial
S = S_initial
for i in range(0,C): 
    print "Cycle " + str(i + 1) + " of " + str(C)
    subDividedEdges = []
    #divides edges in half
    for edge in edges:
        division = []
        for j in range(0, len(edge) - 1 ):
            midPoint = findMidpoint(edge[j],edge[j+1])
            newSubdivision = []
            newSubdivision += [edge[j]]
            newSubdivision += [midPoint]
            division += newSubdivision
        division += [edge[len(edge) -1]]
        subDividedEdges += [division]
    #calculates forces and moves each subdivision point accordingly
    for j in range(0, int(I)):
        if (j+1) % 2 == 0: print " Iteration " + str(j + 1) + " of " + str(int(I))
        newSubDividedEdges = []
        for edge in subDividedEdges: 
            division = []
            division+=[edge[0]]
            kp = K / edgeLength([edge[0],edge[-1]])
            for k in range(1, len(edge) - 1 ):
                force = calculateTotalForces(edge[k-1],edge[k],edge[k+1],k,kp)
                newX = edge[k][0] + S * force[0]
                newY = edge[k][1] + S * force[1]
                division += [[newX,newY]]
            division += [edge[-1]]
            newSubDividedEdges += [division]
        subDividedEdges = newSubDividedEdges
    S = S/2
    I *= I_rate
    edges = subDividedEdges
    

Cycle 1 of 5
 Iteration 2 of 60
 Iteration 4 of 60
 Iteration 6 of 60
 Iteration 8 of 60
 Iteration 10 of 60
 Iteration 12 of 60
 Iteration 14 of 60
 Iteration 16 of 60
 Iteration 18 of 60
 Iteration 20 of 60
 Iteration 22 of 60
 Iteration 24 of 60
 Iteration 26 of 60
 Iteration 28 of 60
 Iteration 30 of 60
 Iteration 32 of 60
 Iteration 34 of 60
 Iteration 36 of 60
 Iteration 38 of 60
 Iteration 40 of 60
 Iteration 42 of 60
 Iteration 44 of 60
 Iteration 46 of 60
 Iteration 48 of 60
 Iteration 50 of 60
 Iteration 52 of 60
 Iteration 54 of 60
 Iteration 56 of 60
 Iteration 58 of 60
 Iteration 60 of 60
Cycle 2 of 5
 Iteration 2 of 40
 Iteration 4 of 40
 Iteration 6 of 40
 Iteration 8 of 40
 Iteration 10 of 40
 Iteration 12 of 40
 Iteration 14 of 40
 Iteration 16 of 40
 Iteration 18 of 40
 Iteration 20 of 40
 Iteration 22 of 40
 Iteration 24 of 40
 Iteration 26 of 40
 Iteration 28 of 40
 Iteration 30 of 40
 Iteration 32 of 40
 Iteration 34 of 40
 Iteration 36 of 40
 Iteration 38 of 40
 I

In [150]:
flatArray = []
maxVal = max(exportVals)
minVal = min(exportVals)
for j in range(0,len(edges)):
    edge = edges[j]
    normalizedVal = (exportVals[j] - minVal) / (maxVal - minVal) 
    flatArray += edge[0]
    #flatArray += [normalizedVal]
    for i in range(1,len(edge) - 1):
        flatArray += edge[i]
        #flatArray += [normalizedVal]
        flatArray += edge[i]
        #flatArray += [normalizedVal]
    flatArray += edge[len(edge) - 1]
    #flatArray += [normalizedVal]
    

In [151]:
for i in range(len(flatArray)):
    flatArray[i] = flatArray[i]*scaling_factor
array.array('f', flatArray).tofile(open('../data/europe2014.bin', 'wb'))

In [132]:
len(edges)

1768